In [7]:
from llama_cpp import Llama
import math
import re
from asteval import Interpreter

In [8]:
llm = Llama.from_pretrained(
	repo_id="TheBloke/phi-2-GGUF",
	filename="phi-2.Q4_K_M.gguf",
    verbose=False,
)

aeval = Interpreter()

llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized


In [ ]:
def run_llama(user_input: str) -> str:
    prompt = f"""You are an AI assistant that never solves math questions itself.
Instead, when a question requires calculation, respond only with:
USE_CALCULATOR: <valid Python expression>

Use Python's math module for functions like square root, sine, cosine, etc.

Examples:
Q: What is 3 + 4?
A: USE_CALCULATOR: 3 + 4

Q: What is the square root of 5?
A: USE_CALCULATOR: math.sqrt(5)

Q: What is 12 * 12 + 10?
A: USE_CALCULATOR: 12 * 12 + 10

Q: What is the sine of 30 degrees?
A: USE_CALCULATOR: math.sin(math.radians(30))

Q: {user_input.strip()}
A:"""

    output = llm(
        prompt,
        max_tokens=100,
        echo=False,
        temperature=0.25,
        stop=["\nQ:"],
    )

    return output['choices'][0]['text'].strip() # type: ignore


    

In [ ]:
def eval_expression(expression: str) -> str:
    print(f"Evaluating expression: {expression}")
    result = aeval(expression)

    if aeval.error:
        print(f"Error evaluating expression: {aeval.error}")
        return f"Error: {aeval.error}"
    
    return result

def parse_response(response):
    print(f"Raw response: {response}")
    match = re.match(r"USE_CALCULATOR:\s*(.+)", response)
    if match:
        expression = match.group(1)
        return f"Result: {eval_expression(expression)}"
    return response


In [11]:
result = run_llama("what is 15 times 34554 divided by 27")

print(parse_response(result))

Raw response: USE_CALCULATOR:
USE_CALCULATOR:
